# 🕸️ Game 1: The Shadow Book

## Interactive Simulation of Network Contagion and Information Asymmetry

---

### 🎯 Risk Focus: *Linkages (L4)* & *Opacity*

This simulation demonstrates how **opacity in private credit markets** creates systemic fragility in the AI "Compute" trade.

---

## The Simulation

You play as a **Chief Risk Officer** managing a portfolio of loans to AI Service Providers (SPVs).

### The Mechanic
- **Limited Visibility**: You can see your own exposure, but you **cannot see the full "Cap Table"** of your borrowers
- **Hidden Network**: Other major lenders ("Whale" funds like Blackstone, Blue Owl, Apollo) also have exposures you can't see
- **Investigation Costs**: You can spend capital to investigate other lenders' positions
- **Liquidity Decisions**: You must decide whether to maintain your loans or recall capital

### The Contagion
If a "Whale" lender takes a loss on a different asset, they panic and de-risk by recalling loans from *your* borrowers—draining liquidity even from healthy companies. **Your portfolio can fail through no fault of your own.**

---

## 🌐 Real-World Analog

### The "Shadow Banking" Information Asymmetry

This simulates the issue flagged by the Bank of England:

- **Private Credit Opacity**: Markets are bilateral and opaque
- **Cross-Collateralization Risk**: A lender doesn't know if their borrower is cross-collateralized by another fund facing a liquidity crisis
- **Network Effects**: In highly connected networks, a shock to one participant becomes a shock to all

### 💡 Thesis Connection

> **"Diversification is an illusion."**

If all lenders are exposed to the same vintage of H100 GPUs, diversification provides no protection. The network linkages mean that:

1. Your borrower may be healthy
2. But another lender's panic on an unrelated position
3. Causes them to pull funding from your borrower
4. Which forces your borrower into default
5. Causing you to take losses

This is **structural contagion** at work.

---

## 🎮 How to Play

### Actions Available Each Turn:

1. **🥶 Hoard (Recall)**: Pull $50M from a target SPV
   - Increases your safe capital
   - BUT damages the SPV's funding health (triggers contagion)
   - May force the SPV into default if funding drops below 70%

2. **🕵️ Investigate**: Spend $10M to learn another lender's position in a target SPV
   - Reveals hidden network exposures
   - Helps you understand contagion risk

3. **⏩ Wait**: Do nothing and advance to the next month
   - Preserves current positions
   - Exposes you to market events

### Win/Loss Conditions:

- **Game Over**: If your capital drops to $0 (fund insolvency)
- **Survival**: Navigate through the crisis while maintaining positive capital

### Key Event:
**Month 3**: The **Blackwell Release** shocks the market, triggering contagion as lenders panic and recall loans

---

## 📊 Portfolio Overview

### Your Initial Exposures:
- **SPV_Core**: $300M
- **SPV_Lambda**: $100M
- **SPV_Crusoe**: $50M

**Total Deployed**: $450M  
**Starting Capital**: $500M  
**Cash Reserve**: $50M

### The Hidden Network:
The exposure matrix shows who lends what to whom—but you can only see YOUR row:

```
                  SPV_Core  SPV_Lambda  SPV_Crusoe
Player_Fund          300        100          50      <- YOU CAN SEE THIS
Blackstone_Bot       500         50          50      <- HIDDEN
BlueOwl_Bot          100        400           0      <- HIDDEN
Apollo_Bot            50          0         400      <- HIDDEN
```

Notice:
- **Blackstone is concentrated in SPV_Core** ($500M)
- **BlueOwl is concentrated in SPV_Lambda** ($400M)
- **Apollo is concentrated in SPV_Crusoe** ($400M)

If any "whale" panics, they'll pull liquidity and trigger cascading defaults.

---

## 🧠 Strategy Tips

1. **Information is Power**: Early investigations can reveal which SPVs are at risk
2. **Liquidity Management**: Balance between keeping loans deployed (earning returns) and hoarding cash (safety)
3. **Contagion Awareness**: Your own actions contribute to contagion—recalling loans damages the network
4. **First-Mover Advantage**: In a crisis, the first to pull capital recovers the most

---

Run the cell below to start the simulation.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import numpy as np

## Game Engine Implementation

The `ShadowNetworkGame` class implements the core simulation logic:

### Key Components:

1. **Exposure Matrix**: Tracks lending positions (mostly hidden from player)
2. **Funding Health**: Measures each SPV's capitalization level (0.0 to 1.0)
3. **Contagion Logic**: Models how withdrawals cascade through the network
4. **Event System**: External shocks (like Blackwell release) trigger panic behavior

### Game State Variables:
- `player_capital`: Your safe capital (starts at $500M)
- `spv_funding_health`: Health of each borrower (1.0 = fully funded, <0.7 = default)
- `matrix`: The exposure matrix (who lends to whom)

In [ ]:
# --- GAME ENGINE ---
class ShadowNetworkGame:
    def __init__(self):
        self.turn = 1
        self.player_capital = 500
        self.is_alive = True
        self.logs = []
        
        # Players & Assets
        self.lenders = ["Player_Fund", "Blackstone_Bot", "BlueOwl_Bot", "Apollo_Bot"]
        self.spvs = ["SPV_Core", "SPV_Lambda", "SPV_Crusoe"]
        
        # Exposure Matrix (Hidden State)
        # Rows = Lenders, Cols = Borrowers
        data = [
            [300, 100, 50],   # You
            [500, 50,  50],   # Blackstone (The Whale)
            [100, 400, 0],    # BlueOwl
            [50,  0,   400]   # Apollo
        ]
        self.matrix = pd.DataFrame(data, index=self.lenders, columns=self.spvs)
        self.spv_funding_health = {spv: 1.0 for spv in self.spvs} # 1.0 = 100% Funded

    def next_turn(self, action, target_spv=None):
        if not self.is_alive: return
        self.logs = []
        
        # 1. Player Action
        if action == "Hoard (Recall)":
            exposure = self.matrix.loc["Player_Fund", target_spv]
            if exposure > 0:
                pulled = 50
                self.matrix.loc["Player_Fund", target_spv] -= pulled
                self.player_capital += pulled
                self.log(f"🥶 YOU: Recalled ${pulled}M from {target_spv}. (Triggered Contagion)")
                
                # Apply Damage (Self-inflicted contagion)
                current_total = self.matrix[target_spv].sum() + 50
                damage_pct = pulled / current_total
                self.spv_funding_health[target_spv] -= damage_pct

        elif action == "Investigate":
            self.player_capital -= 10
            bot = "Blackstone_Bot" 
            pos = self.matrix.loc[bot, target_spv]
            self.log(f"🕵️ INTEL: {bot} has ${pos}M in {target_spv}.")

        # 2. Events (The Blackwell Shock)
        if self.turn == 3:
            shock_target = "SPV_Core"
            self.log(f"🚨 EVENT: Blackwell Release! {shock_target} devalued.")
            
            # Simulated Contagion: BlueOwl gets scared of Core, pulls from Lambda
            self.spv_funding_health["SPV_Core"] -= 0.30
            self.spv_funding_health["SPV_Lambda"] -= 0.15 
            self.log("📉 CONTAGION: BlueOwl_Bot panicked and pulled funds from SPV_Lambda.")

        # 3. Check Survival
        for spv in self.spvs:
            if self.spv_funding_health[spv] < 0.70:
                loss = self.matrix.loc["Player_Fund", spv]
                if loss > 0:
                    self.player_capital -= loss
                    self.matrix.loc["Player_Fund", spv] = 0
                    self.log(f"☠️ DEFAULT: {spv} Collapsed! You lost ${loss}M.")
        
        if self.player_capital <= 0:
            self.is_alive = False
            self.log("GAME OVER: Fund Insolvent.")
            
        self.turn += 1

    def log(self, msg):
        self.logs.append(f"Month {self.turn}: {msg}")

## User Interface Setup

The cells below create an interactive dashboard with:
- Real-time portfolio view
- Color-coded health indicators
- Action buttons for gameplay
- Activity log showing recent events

In [ ]:
# --- UI LOGIC ---
game = ShadowNetworkGame()
out = widgets.Output()

def render_ui():
    with out:
        clear_output()
        if not game.is_alive:
            print("💀 GAME OVER: INSOLVENCY")
            for l in game.logs: print(l)
            return

        # Dashboard
        display(HTML(f"<h3>🗓 Month: {game.turn} | 💰 Capital: ${game.player_capital}M</h3>"))
        
        # Portfolio View
        my_pos = game.matrix.loc["Player_Fund"]
        health_data = pd.DataFrame({
            "My Exposure": my_pos,
            "Funding Health": [f"{game.spv_funding_health[spv]*100:.0f}%" for spv in game.spvs]
        })
        
        # Color coding function
        def color_health(val):
            val = int(val.replace("%",""))
            if val < 80: return 'color: red; font-weight: bold'
            return 'color: green'

        display(health_data.style.applymap(color_health, subset=["Funding Health"]))
        
        print("\n--- ACTIVITY LOG ---")
        for l in game.logs[-5:]: print(l)

## Event Handlers & Game Controls

This cell sets up the interactive buttons and connects them to game actions.

In [ ]:
# Handlers
def on_hoard(b):
    game.next_turn("Hoard (Recall)", dropdown.value)
    render_ui()

def on_scan(b):
    game.next_turn("Investigate", dropdown.value)
    render_ui()

def on_wait(b):
    game.next_turn("Wait")
    render_ui()

# Widgets
dropdown = widgets.Dropdown(options=["SPV_Core", "SPV_Lambda", "SPV_Crusoe"], description="Target:")
btn_hoard = widgets.Button(description="🥶 Hoard Loan", button_style='danger')
btn_scan = widgets.Button(description="🕵️ Investigate", button_style='info')
btn_wait = widgets.Button(description="⏩ Wait")

btn_hoard.on_click(on_hoard)
btn_scan.on_click(on_scan)
btn_wait.on_click(on_wait)

## 🎮 START THE GAME

Run the cell below to begin the simulation. Use the dropdown to select a target SPV and the buttons to take actions.

**Remember**: 
- Month 3 brings the Blackwell shock
- SPVs default when funding health drops below 70%
- You can investigate to reveal hidden exposures
- But information costs capital ($10M per investigation)

Good luck! 🍀

In [ ]:
render_ui()
display(out)
display(widgets.HBox([dropdown]))
display(widgets.HBox([btn_hoard, btn_scan, btn_wait]))

---

## �� Lessons from The Shadow Book

### What This Simulation Teaches:

1. **Opacity Creates Systemic Risk**
   - You cannot protect against risks you cannot see
   - Private credit markets lack transparency into network linkages

2. **Diversification Fails in Connected Networks**
   - Your three SPVs appear diversified
   - But shared lender exposure creates hidden correlation
   - When one "whale" panics, all SPVs suffer

3. **Contagion is Structural, Not Fundamental**
   - SPVs can be operationally healthy
   - But fail due to liquidity withdrawal from unrelated shocks
   - The failure mode is **funding**, not **operations**

4. **First-Mover Advantage Accelerates Crises**
   - Rational players have incentive to pull capital early
   - This creates a self-fulfilling prophecy
   - Cooperative equilibrium is unstable

### Real-World Implications:

In the AI compute financing market:
- Large private credit funds (Blackstone, Blue Owl, Apollo, Ares) all lend to GPU-backed SPVs
- These positions are not disclosed publicly
- A shock to GPU valuations (e.g., next-gen chip release) affects ALL lenders simultaneously
- The first to exit recovers capital; the last faces total loss
- This structure guarantees **fire sales** during stress

---

### 🔗 Connection to Other Simulations:

- **Game 2 (The Syndicate)**: Explores the prisoner's dilemma dynamics when lenders know they're interconnected
- **Game 3 (Tranche Defense)**: Shows how individual SPV capital structures fail under asset depreciation

Together, these three games demonstrate the "Four Ls" framework of systemic risk in AI compute financing.